<a href="https://colab.research.google.com/github/with2say/layout2inspection/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/with2say/layout2inspection.git

Cloning into 'layout2inspection'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), 7.31 KiB | 1.83 MiB/s, done.


In [ ]:
!pip install pytorch-lightning --quiet

In [ ]:
import sys
sys.path.append("layout2inspection")

from utils import *
from dataset import *
from model import *
from polytoimage import *


def main(n_samples=10000, n_epoch=200, d_model=16, nhead=8, out_h=16, out_w=16):
    # input params
    n_samples = n_samples
    n_channels = 2
    range_shape = [1, 3]
    range_polygon = [3, 5]
    n_positions = 2
    n_outputs = 1

    # checkpoint_path = '/workspaces/layout2inspection/lightning_logs/version_15/checkpoints/epoch=21-step=704.ckpt'

    # addtional params
    n_shapes = np.max(range_shape)
    n_polygons = np.max(range_polygon)

    # hyper params
    # data, targets = generate_data_with_negative_padding(n_samples, range_shape, range_polygon)
    data, targets = generate_dataset(n_samples, n_channels, range_shape, range_polygon)
    print('null mse:', np.var(targets))
    print(data.shape, targets.shape)

    data_module = PolygonAreaDataModule(data, targets, batch_size=128, val_split=0.1, test_split=0.1, num_workers=2)

    # MultiShapeEmbedding 객체 생성
    d_model = d_model
    nhead = nhead
    out_h = out_h
    out_w = out_w
    layer = MultiShapeEmbedding(
        n_positions, n_polygons, n_shapes, n_channels, n_outputs,
        d_model, nhead, out_h, out_w
    )

    # Create the model
    # layer = Polygons2Area(d_model=64,
    #                       nhead=16,
    #                       num_layers=16,
    #                       num_position=num_position,
    #                       num_polygons=np.max(polygon_range),
    #                       dropout=0.01)

    # d_model = num_position
    # nhead = 2
    # num_layers = 2
    # dim_feedforward = 4
    # layer = HierarchicalTransformer(d_model=num_position,
    #                                 nhead=nhead,
    #                                 num_layers=num_layers,
    #                                 dim_feedforward=dim_feedforward)

    model = PolygonRegressor(layer)
    # model.load_from_checkpoint(checkpoint_path)


    # trainer = pl.Trainer(max_epochs=n_epoch)
    trainer = get_trainer(n_epoch)
    trainer.fit(model, data_module)
    trainer.validate(model, datamodule=data_module)

    y_true, y_pred = get_predictions(trainer, model, data_module)
    print(np.shape(y_true), np.shape(y_pred))
    evaluate_regression(y_true, y_pred)

    plot_true_vs_predicted(y_true, y_pred)



if __name__ == '__main__':
    main(
        n_samples=20000, 
        n_epoch=300, 
        d_model=128,
        nhead=32,
        out_h=32,
        out_w=32,
    )
    

In [ ]:
from tutorial import main
main(
    n_samples=20000, 
    n_epoch=300, 
    d_model=128,
    nhead=32,
    out_h=32,
    out_w=32,
    )

